In [24]:
import pandas as pd
import psycopg2

In [25]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [26]:
DBNAME = "opportunity_youth"

In [27]:
conn = psycopg2.connect(dbname=DBNAME)
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = []
for table in cursor.fetchall():
    tables.append(table[0])
tables

['pums_2017',
 'puma_names_2010',
 'wa_jobs_2017',
 'wa_geo_xwalk',
 'ct_puma_xwalk']

In [28]:
pums_2017_df = pd.read_sql("SELECT * FROM pums_2017;", conn)
puma_names_2010_df = pd.read_sql("SELECT * FROM puma_names_2010;", conn)
wa_jobs_2017_df = pd.read_sql("SELECT * FROM wa_jobs_2017;", conn)
wa_geo_xwalk_df = pd.read_sql("SELECT * FROM wa_geo_xwalk;", conn)
ct_puma_xwalk_df = pd.read_sql("SELECT * FROM ct_puma_xwalk;", conn)

In [23]:
df_new = pd.read_sql("""
SELECT sporder, puma, agep, cow, wkl, sch, schl, rt
FROM pums_2017
WHERE (agep BETWEEN 16.0 AND 24.9)
AND CAST(sch AS int) = 1
AND CAST(cow AS int) = 9
""", conn)
df_new


,sporder,puma,agep,cow,wkl,sch,schl,rt
0,02,10502,23.0,9,3,1,13,P
1,02,11503,24.0,9,3,1,22,P
2,02,10502,19.0,9,3,1,16,P
3,03,10400,19.0,9,3,1,16,P
4,02,11503,19.0,9,3,1,16,P
...,...,...,...,...,...,...,...,...
269,04,11502,20.0,9,3,1,16,P
270,01,11605,23.0,9,3,1,19,P
271,02,10503,18.0,9,3,1,16,P
272,06,11505,21.0,9,3,1,13,P


In [15]:
pums_2017.shape

NameError: name 'pums_2017' is not defined

In [36]:
print(pums_2017_df.cow.dtype)

object


In [48]:
df_new.info

<bound method DataFrame.info of       sporder   puma  agep   cow schl  ssp     wagp  wkhp wkl  st
0          03  11505  18.0     3   14  0.0    500.0   3.0   1  53
1          02  10300  17.0  None   14  0.0      0.0   NaN   3  53
2          02  11801  17.0     1   16  0.0   3700.0  20.0   1  53
3          01  10100  21.0     1   20  0.0   1800.0  40.0   1  53
4          02  10800  23.0     1   16  0.0   9000.0  50.0   1  53
...       ...    ...   ...   ...  ...  ...      ...   ...  ..  ..
38165      01  11602  19.0     1   16  0.0   2100.0  30.0   1  53
38166      01  11701  21.0     1   14  0.0   8000.0  28.0   1  53
38167      01  11801  22.0     5   16  0.0  24000.0  70.0   1  53
38168      01  11602  19.0     1   18  0.0    300.0  12.0   1  53
38169      01  10600  19.0     1   16  0.0   2500.0  30.0   1  53

[38170 rows x 10 columns]>

In [49]:
df_new.cow.isna().sum()

10074

In [56]:
#list of Employment Status (COW), School Status(SCH), NWAV (Availble for work), NWAB (Temporary absence)
#NWLA (On lay off), NLWK (Looking for work).
#Table to view NULL values, to allow us to add filters to include more people.
df_cs = pd.read_sql("""
SELECT cow, sch, nwav, nwab, nwla, nwlk
FROM pums_2017
;""", conn)
df_cs

,cow,sch,nwav,nwab,nwla,nwlk
0,None,1,5,2,2,2
1,None,1,5,2,2,2
2,None,1,5,2,2,2
3,None,1,5,2,2,2
4,1,1,5,3,3,3
...,...,...,...,...,...,...
359070,5,1,5,3,3,3
359071,1,3,5,3,3,3
359072,1,2,5,2,2,2
359073,None,1,5,2,2,2


In [57]:
df_cs.describe()

,cow,sch,nwav,nwab,nwla,nwlk
count,215545,347443,291865,291865,291865,291865
unique,9,3,5,3,3,3
top,1,1,5,3,3,3
freq,132602,264532,266800,172064,166563,172483


In [70]:
#df_cs, but with filters. Having issues with NWAB/NWLA/NWLK setting them to = 1 OR 3
#NEED TO FIX THE FORMATTING, so its agep and (cow or nwab or nwla or nwlk)
df_cs_fil = pd.read_sql("""
SELECT cow, nwav, nwab, nwla, nwlk
FROM pums_2017
WHERE (agep BETWEEN 16.0 AND 24.9)
AND CAST(cow AS int) = 9
OR CAST(nwab as int) = 3
OR CAST(nwla as int) = 3
OR CAST(nwlk as int) = 3
;""", conn)
df_cs_fil

,cow,nwav,nwab,nwla,nwlk
0,1,5,3,3,3
1,3,5,3,3,3
2,3,5,3,3,3
3,1,5,3,3,3
4,1,5,3,3,3
...,...,...,...,...,...
178550,6,5,3,3,3
178551,1,5,3,3,3
178552,None,5,3,3,3
178553,1,5,3,3,3


In [71]:
df_cs_fil.describe()

,cow,nwav,nwab,nwla,nwlk
count,165703,178555,178555,178555,178555
unique,9,5,3,3,3
top,1,5,3,3,3
freq,102589,173980,172064,166563,172483


In [72]:
#NAN Count for df_cs_fil
len(df_cs_fil) - df_cs_fil.count()

cow     12852
nwav        0
nwab        0
nwla        0
nwlk        0
dtype: int64